In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

In [2]:
from googletrans import Translator
import re

In [3]:
def backtranslate_with_placeholders(text, src_lang='ko', target_lang='en'):
    text = text.split('\n')
    length = len(text)

    translator = Translator()

    result = ''

    for i in range(length):
        person = text[i][:11]
        dia = text[i][11:]
        try:
            translate_text = translator.translate(dia, src=src_lang, dest=target_lang).text
            back_translate_text = translator.translate(translate_text, src=target_lang, dest=src_lang).text
        except Exception as e:
            print(f"An error occurred during translation: {e}")
            return -1
        test_result = person + back_translate_text  
        
        if i != length - 1:
            test_result = test_result + '\n'

        result = result + test_result

    return result
        
input_text = "#Person1#: 안녕하세요, 박씨, 어떻게 지내세요?\n#Person2#: 안녕하세요, 피터스 박사님. 안녕하세요, 감사합니다. 리키와 저는 예방 접종을 받으러 여기 왔어요.\n#Person1#: 알겠어요. 리키의 예방 접종 이력을 보니 이미 소아마비, 딕테리아 및 B형간염 예방접종을 받았네요. 그는 14개월이니 이제 A형간염, 수두 및 홍역 예방접종을 받아야 해요.\n#Person2#: 풍진 및 볼거리는 어떻게 되죠?\n#Person1#: 이 백신들은 지금 받을 수 있고 나머지는 몇 주 후에 받아야 해요.\n#Person2#: 알겠어요, 박사님, 딕테리아 백신도 필요할 것 같아요. 마지막으로 맞은 게 15년 전이에요!\n#Person1#: 당신의 기록을 확인하고 간호사에게 부스터를 접종하도록 하겠습니다. 이제 리키의 팔을 꽉 잡아 주세요. 조금 따가울 수 있어요."
backtranslated_text = backtranslate_with_placeholders(input_text)
backtranslated_text

'#Person1#: 안녕하세요, 박 씨, 잘 지내세요?\n#Person2#: 안녕하세요, 피터스 박사님 헬로 감사합니다.\n#Person1#: Ricky의 예방 접종 기록은 이미 소아마비, Dicks 및 B 형 간염에 대한 예방 접종을 받았으며, 14 개월이므로 A 형 간염, 수두 및 홍역을 위해 예방 접종을 받아야합니다.\n#Person2#: 풍진과 명소는 무엇입니까?\n#Person1#: 이 백신은 지금받을 수 있으며 나머지는 몇 주 안에 받아야합니다.\n#Person2#: 좋아, 박사님, 독재 백신이 필요하다고 생각합니다. 15 년 전이었습니다.\n#Person1#: 나는 당신의 기록을 확인하고 간호사에게 부스터를 접종 할 것입니다.'

In [4]:
df = pd.read_csv("/data/ephemeral/home/data/train.csv")

In [5]:
# for i in range(len(df)):
#     fname = f'train_{i}'
#     df.loc[i, 'fname'] = fname

In [6]:
# df.to_csv("/data/ephemeral/home/data/train.csv", index=False)

In [7]:
df

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12452,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12453,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12454,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12455,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [8]:
s_idx = len(df)
except_list = dict()


for i in tqdm(range(len(df))):
    idx = s_idx + i
    df.loc[idx, 'fname'] = f'train_{idx}'
    df.loc[idx, 'topic'] = df.loc[i, 'topic']
    df.loc[idx, 'summary'] = df.loc[i, 'summary']
    dia = backtranslate_with_placeholders(text=df.loc[i, 'dialogue'])
    if dia != -1 :
        df.loc[idx, 'dialogue'] = dia 
    else:
        except_list[idx] = i
        print(f'index: {i}에서 google translate api 오류 발생 !!!')
    

  0%|          | 49/12457 [12:36<51:42:46, 15.00s/it]

An error occurred during translation: The read operation timed out
index: 48에서 google translate api 오류 발생 !!!


  1%|          | 116/12457 [35:08<60:58:26, 17.79s/it]

An error occurred during translation: The read operation timed out
index: 115에서 google translate api 오류 발생 !!!


  1%|          | 122/12457 [37:40<75:39:11, 22.08s/it]

...

 21%|██        | 2611/12457 [14:44:59<41:30:35, 15.18s/it]

An error occurred during translation: The read operation timed out
index: 2610에서 google translate api 오류 발생 !!!


 21%|██        | 2621/12457 [14:47:34<33:23:10, 12.22s/it]

An error occurred during translation: The read operation timed out
index: 2620에서 google translate api 오류 발생 !!!


 22%|██▏       | 2710/12457 [15:16:11<39:06:34, 14.44s/it]

In [ ]:
df.to_csv("/data/ephemeral/home/data/train_add_translate.csv", index=False)

In [ ]:
except_df = pd.Dataframe(columns=['key', 'value'])
index = 0
for k, v in except_list.items():
    except_df.loc[index, 'key'] = k 
    except_df.loc[index, 'value'] = v
    index += 1

except_df.to_csv("/data/ephemeral/home/data/except_list.csv", index=False)